In [11]:
import pandas as pd
import numpy as np
import PIL
from keras.applications import VGG16,imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
from keras.layers import Dense
import numpy as np
import glob
import os
from IPython.display import Image,display

import pickle

In [2]:
df = pd.read_table('Data/DeepFashion/list_attr_img.txt',skiprows=1,sep='\s+',header=None)

In [3]:
bbox = pd.read_table('Data/DeepFashion/list_bbox.txt',sep='\s+')

In [4]:
joined = bbox.join(df,lsuffix='image_name',rsuffix='0')

In [5]:
joined.shape[0]

289222

In [6]:
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input

def preprocess_DF(bbox):
    image = crop_resize_DF(bbox)
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    
    return preprocess(image)

def crop_resize_DF(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name'])
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    
    img = img.crop((lx,ly,ux,uy))
    img = img.resize(inputShape, PIL.Image.ANTIALIAS)
    return img

In [ ]:
import time
N = 20000
subset = np.random.randint(joined.shape[0],size=N)
proc_imgs = []
start = time.time()

for i in subset:
    proc_imgs.append(np.squeeze(preprocess_DF(joined.iloc[i])))
proc_imgs = np.array(proc_imgs)
print(f'{time.time()-start:.2f} s')

In [12]:
def generator(df, batch_size):
    subset = np.random.randint(joined.shape[0],size=batch_size)
    proc_imgs = []
 
    while True:
        for i in range(batch_size):
         # choose random index in features
         proc_imgs.append(np.squeeze(preprocess_DF(df.iloc[i])))
    
    attr = (df.iloc[subset,6:].values>0)
    yield np.array(proc_imgs), attr

In [ ]:
joined.head()

In [8]:
attr = (joined.iloc[:,6:].values>0)

In [9]:
class_weights = 1/(np.mean(attr,axis=0)*1000+1e-8)

In [10]:
base_model = VGG16(weights='imagenet')
x = base_model.get_layer('predictions').input
predictions = Dense(1000, activation='sigmoid',name='predictions')(x)

new_model = Model(inputs=base_model.input, outputs=predictions)

new_model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
from keras import callbacks
tbCB = callbacks.TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=False, write_images=False)
checkpointCB = callbacks.ModelCheckpoint("checkpoints/{}".format(time()))

new_model.fit_generator(generator(,verbose=1,callbacks=[tbCB,checkpointCB])

In [79]:
attr.shape

(10000, 1000)